In [1]:
import cv2
import numpy as np

In [2]:
# reading image and converting it to grayscale
img = cv2.imread('test_images/test_image_1.png', cv2.IMREAD_GRAYSCALE)
  
# # converting image into grayscale image
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

Converting to grayscale allows for easy detection of objects, but still different colours give different gradients of gray, so it is necessary to threshold them to find contours.

Thresholding is a very popular segmentation technique, used for separating an object considered as a foreground from its background.

**cv2.threshold**

First argument is the source image, which should be a grayscale image. Second argument is the threshold value which is used to classify the pixel values. Third argument is the maxVal which represents the value to be given if pixel value is more than the threshold value. OpenCV provides different styles of thresholding and it is decided by the fourth parameter of the function.


*cv2.THRESH_BINARY*: If pixel intensity is greater than the set threshold, value set to 255, else set to 0 (black).

**cv2.threshold(source, thresholdValue, maxVal, thresholdingTechnique)** 

Parameters: 

-> source: Input Image array (must be in Grayscale). 

-> thresholdValue:(all pixel values above this value will be changed to white(255), whereas other values to be changed to black(0)).

-> maxVal: Maximum value that can be assigned to a pixel. 

-> thresholdingTechnique: The type of thresholding to be applied. 

cv2.threshold returns two values, only one is needed for shape detection

In [3]:
_, threshold = cv2.threshold(img, 200, 255, cv2.THRESH_BINARY)

Contours can be explained simply as a curve joining all the continuous points (along the boundary), having same color or intensity.


There are three arguments in **cv2.findContours()** function, first one is source image, second is contour retrieval mode, third is contour approximation method. And it outputs the contours and hierarchy.


The mode *cv2.RETR_TREE* finds all the promising contour lines and reconstructs a full hierarchy of nested contours. The method *cv2.CHAIN_APPROX_SIMPLE* returns only the endpoints that are necessary for drawing the contour line. It removes all redundant points and compresses the contour, thereby saving memory (as compared to *cv2.CHAIN_APPROX_NONE*, which stores all points)

In [4]:
contours,_ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

**cv2.drawContours**

First argument is source and destination image, second argument is the contours which should be passed as a Python list, third argument is index of contours (useful when drawing individual contour. To draw all contours, pass -1) and remaining arguments are color, thickness etc.

We'll count the vertices as the intersecting points of the contours to determine the shape.



This may have errorenous contours near the vertices, so we use approximation.
**cv2.approxPolyDP(input_curve, epsilon, closed)**
It approximates a contour shape to another shape with less number of vertices depending upon the precision we specify.
Second argument is called epsilon, which is maximum distance from contour to approximated contour. It is an accuracy parameter. A wise selection of epsilon is needed to get the correct output. The third argument is True, which indicates that the shape is closed.


**cv2.arcLength()** is used to calculate the perimeter of the contour(cnt) and takes 1% (0.01) of this length for cv2.approxPolyDP as epsilon. If the second argument is True then it considers the contour to be closed. 

In [5]:
#to loop over the contours
i=0
for cnt in contours:
    
    # here we are ignoring first counter because 
    # findcontour function detects whole image as shape
    if i==0:
        i=1
        continue
        
    # using drawContours() function (non-approximated)
    #cv2.drawContours(img, [contour], 0, (0, 0, 255), 5)
    
    #approximating
    approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt, True), True)
    
    cv2.drawContours(img, [approx], 0, (0), 3)
    
    # printing the values in approx
    # print(len(approx))
    
    # finding out the top coordinates of each shape
    x = approx.ravel()[0]
    y = approx.ravel()[1]
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    # putting shape name at top of each shape
    if len(approx) == 3:
        cv2.putText(img, 'Triangle', (x, y), font, 0.5, (0, 0, 0))
  
    elif len(approx) == 4:
        
        # for distinguishing between rectangle and square
        x, y, w, h = cv2.boundingRect(approx)

        aspectRatio = float (w)/h
        if aspectRatio >= 0.95 and aspectRatio <= 1.05:
            cv2.putText(img, 'Square', (x, y), font, 0.5, (0, 0, 0))
        else:
            cv2.putText(img, 'Rectangle', (x, y), font, 0.5, (0, 0, 0))
    
    elif len(approx) == 5:
        cv2.putText(img, 'Pentagon', (x, y), font, 0.5, (0, 0, 0))
  
    else:
        cv2.putText(img, 'Circle', (x, y), font, 0.5, (0, 0, 0))
        
    # finding center point of shape
    M = cv2.moments(cnt)
    if M['m00'] != 0.0:
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
    print(cx,cy)
  

588 370
325 145


**approx.ravel()**

If we just print approx it'll print an array of array of the coordinates of the polygon, ravel() function shortens it to a single array. For each iteration of the for loop the first coordinates (x and y) are given by approx.ravel()[0] and approx.ravel()[1].



**cv2.putText(image, text, org, font, fontScale, color[, thickness[, lineType[, bottomLeftOrigin]]])**

image: It is the image on which text is to be drawn.

text: Text string to be drawn.

org: It is the coordinates of the bottom-left corner of the text string in the image. The coordinates are represented as tuples of two values i.e. (X coordinate value, Y coordinate value).

font: It denotes the font type. Some of font types are FONT_HERSHEY_SIMPLEX, FONT_HERSHEY_PLAIN, , etc.

fontScale: Font scale factor that is multiplied by the font-specific base size.

color: It is the color of text string to be drawn. For BGR, we pass a tuple. eg: (0, 0, 0) for black color.

thickness: It is the thickness of the line in px.

lineType: This is an optional parameter.It gives the type of the line to be used.

bottomLeftOrigin: This is an optional parameter. When it is true, the image data origin is at the bottom-left corner. Otherwise, it is at the top-left corner.



**cv2.boundingRect()**

It is a function used to create an approximate rectangle along with the image. This function’s primary use is to highlight the area of interest after obtaining the image’s outer shape. With proper markings, the users can easily highlight the desired aspect in an image.

Let (x,y) be the top-left coordinate of the rectangle and (w,h) be its width and height.
aspectRatio is the ratio of width (w) by height (h). Ideally, for a square, its aspect ratio is 1, but we do incorporate some noises that may have crept in by taking it between 0.95 and 1.05.

Image Moment is a particular weighted average of image pixel intensities, with the help of which we can find some specific properties of an image, like radius, area, centroid etc. To find the centroid of the image, we generally convert it to binary format and then find its center.

**cv2.moments()**

cv2.threshold returns two values, only one is needed for shape detection

In [ ]:
cv2.imshow("shapes", img)
cv2.imshow("Thresholded", threshold)
cv2.waitKey(0)
cv2.destroyAllWindows()